<a href="https://colab.research.google.com/github/sanjayakanungo/RAG/blob/main/Customize_RAG_Pipeline_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Pipeline:

*   Configure different LLM.
*   Use a Embedding Model.
*   Configure Vector Store.
*   Customize with different Indices.
*   Synthesize response for a query.




In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install llama-index qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.7/630.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#!pip install python-dotenv --quiet
!pip install python-dotenv
import dotenv
import os
dotenv.load_dotenv(
        os.path.join('/content/drive/MyDrive/', '.env')
    )
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


Load Data

In [5]:
!pip install pypdf
#from pathlib import Path
#from llama_index import download_loader, SimpleDirectoryReader
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("/content/drive/MyDrive/GENAI-Pinnacle/VCFdataset").load_data()

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
len(documents)

771

In [7]:
print(documents[10].text)

About the VMware Cloud Foundation 
Administration Guide
The VMware Cloud Foundation  Administration Guide  provides information about managing a 
VMware Cloud Foundation ™ system, including managing the system's virtual infrastructure, 
managing users, configuring, upgrading, and monitoring the system. 
Intended Audience
The VMware Cloud Foundation  Administration Guide  is intended for cloud architects, 
infrastructure administrators, and cloud administrators who are familiar with and want to use 
VMware software to quickly deploy and manage a software-defined data center (SDDC). The 
information in this document is written for experienced data center system administrators who 
are familiar with:
nConcepts of virtualization, software-defined data centers, and virtual infrastructure (VI)
nVMware virtualization technologies, such as VMware ESXi ™, the hypervisor
nSoftware-defined networking using VMware NSX®
nSoftware-defined storage using VMware vSAN ™
nNetworking concepts such as Laye

Configure OpenAI LLM

In [10]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

Load BGE embeddings from HuggingFace

In [16]:
!pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Create Service Context by providing LLM and Embedding model



In [18]:
from llama_index.core import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

<ipython-input-18-0b686f76eddf>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


Configure Qdrant VectorDB

In [20]:
!pip install llama-index-vector-stores-qdrant
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# initialize client, setting path to save data
client = qdrant_client.QdrantClient(path="/content/drive/MyDrive/GENAI-Pinnacle/qdrant_db")

# create collection
vector_store = QdrantVectorStore(client=client, collection_name="rag_customization")

Create Store Context by assigning vector store created

In [22]:
#from llama_index.storage.storage_context import StorageContext
from llama_index.core import StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#1.VectorStore Index

Define the vector store index by passing storage context and service context

In [24]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents,
    storage_context=storage_context,
    service_context=service_context,
    show_progress=True
 )

Parsing nodes:   0%|          | 0/771 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/771 [00:00<?, ?it/s]

Build the query engine for the index

In [25]:
query_engine = index.as_query_engine(similarity_top_k=10)

In [26]:
response = query_engine.query(
    "What are the Physical Network Design Requirements for VMware Cloud Foundation?"
)

In [27]:
print(response)

The Physical Network Design Requirements for VMware Cloud Foundation are not explicitly mentioned in the given context.
